In [ ]:
#| default_exp models.gru

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

#  GRU

Cho et. al proposed the Gated Recurrent Unit (`GRU`) to improve on LSTM and Elman cells. The predictions at each time are given by a MLP decoder. This architecture follows closely the original Multi Layer Elman `RNN` with the main difference being its use of the GRU cells.

**References**<br>
-[Junyoung Chung, Caglar Gulcehre, KyungHyun Cho, Yoshua Bengio (2014). "Empirical Evaluation of Gated Recurrent Neural Networks on Sequence Modeling".](https:arxivorg/abs/1412.3555)<br>
-[Kyunghyun Cho, Bart van Merrienboer, Dzmitry Bahdanau, Yoshua Bengio (2014). "On the Properties of Neural Machine Translation: Encoder-Decoder Approaches".](https://arxiv.org/abs/1409.1259)<br>

![Figure 1. Gated Recurrent Unit Cell.](imgs_models/gru.png)

In [ ]:
#| hide
from nbdev.showdoc import show_doc

In [ ]:
#| export
import torch
import torch.nn as nn

from neuralforecast.losses.pytorch import MAE
from neuralforecast.common._base_recurrent import BaseRecurrent

In [ ]:
#| export
class GRU(BaseRecurrent):
    """ GRU

    Multi Layer Recurrent Network with Gated Units (GRU), and simple multi-step 
    MLP decoder. The network has `tanh` or `relu` non-linearities, it is trained 
    using ADAM stochastic gradient descent. The network accepts static, historic 
    and future exogenous data, flattens the inputs.

    **Parameters:**<br>
    `h`: int, forecast horizon.<br>
    `input_size`: int, autorregresive inputs size, y=[1,2,3,4] input_size=2 -> y_[t-2:t]=[1,2].<br>
    `n_layers`: int=2, number of layers for the GRU.<br>
    `state_hsize`: int=200, units for the GRU's hidden state size.<br>
    `activation`: str=`tanh`, type of GRU activation from `tanh` or `relu`.<br>
    `bias`: bool=True, whether or not to use biases b_ih, b_hh within GRU units.<br>
    `dropout`: float=0., dropout regularization applied to GRU outputs.<br>
    `stat_exog_list`: str list, static exogenous columns.<br>
    `hist_exog_list`: str list, historic exogenous columns.<br>
    `futr_exog_list`: str list, future exogenous columns.<br>
    `loss`: PyTorch module, instantiated train loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
    `learning_rate`: float=1e-3, initial optimization learning rate (0,1).<br>
    `batch_size`: int=32, number of differentseries in each batch.<br>
    `step_size`: int=1, step size between each window of temporal data.<br>
    `scaler_type`: str='robust', type of scaler for temporal inputs normalization see [temporal scalers](https://nixtla.github.io/neuralforecast/common.scalers.html).<br>
    `random_seed`: int=1, random_seed for pytorch initializer and numpy generators.<br>
    `num_workers_loader`: int=os.cpu_count(), workers to be used by `TimeSeriesDataLoader`.<br>
    `drop_last_loader`: bool=False, if True `TimeSeriesDataLoader` drops last non-full batch.<br>
    `**trainer_kwargs`: int,  keyword trainer arguments inherited from [PyTorch Lighning's trainer](https://pytorch-lightning.readthedocs.io/en/stable/api/pytorch_lightning.trainer.trainer.Trainer.html?highlight=trainer).<br>    
    """
    def __init__(self,
                 h: int,
                 input_size: int,
                 n_layers: int = 2,
                 state_hsize: int = 200, 
                 activation: str = 'tanh',
                 bias: bool = True,
                 dropout: float = 0.,
                 futr_exog_list = None,
                 hist_exog_list = None,
                 stat_exog_list = None,
                 loss=MAE(),
                 learning_rate: float = 1e-3,
                 batch_size=32,
                 step_size: int = 1,
                 scaler_type: str='robust',
                 num_workers_loader=0,
                 drop_last_loader=False,
                 random_seed=1,
                 **trainer_kwargs):
        super(GRU, self).__init__(
            h = h,
            input_size = input_size,
            loss=loss,
            learning_rate = learning_rate,
            batch_size=batch_size,
            scaler_type=scaler_type,
            futr_exog_list=futr_exog_list,
            hist_exog_list=hist_exog_list,
            stat_exog_list=stat_exog_list,
            num_workers_loader=num_workers_loader,
            drop_last_loader=drop_last_loader,
            random_seed=random_seed,
            **trainer_kwargs
        )

        # Architecture
        self.state_hsize = state_hsize
        self.step_size = step_size
        self.n_layers = n_layers
        self.activation = activation
        self.bias = bias
        self.dropout = dropout

        self.futr_exog_size = len(self.futr_exog_list)
        self.hist_exog_size = len(self.hist_exog_list)
        self.stat_exog_size = len(self.stat_exog_list)

        input_rnn = input_size + self.hist_exog_size*input_size + \
                    self.futr_exog_size*(input_size + h) + self.stat_exog_size

        # Instantiate model
        self.gru = nn.GRU(input_size=input_rnn,
                            hidden_size=self.state_hsize,
                            num_layers=self.n_layers,
                            bias=self.bias,
                            dropout=self.dropout,
                            batch_first=True)
        self.adapterW  = nn.Linear(self.state_hsize, self.h)

    def forward(self, windows_batch):
        
        # Parse windows_batch
        insample_y    = windows_batch['insample_y']
        futr_exog     = windows_batch['futr_exog']
        hist_exog     = windows_batch['hist_exog']
        stat_exog     = windows_batch['stat_exog']

        # Flatten inputs [B, W, C, L+H] -> [B, W, C*(L+H)]
        # Contatenate [ Y_t, | X_{t-L},..., X_{t} | F_{t-L},..., F_{t+H} | S ]
        batch_size, windows_size = insample_y.shape[:2]
        if self.hist_exog_size > 0:
            hist_exog = hist_exog.permute(0,2,1,3) # [B, C, W, L] -> [B, W, C, L]
            insample_y = torch.cat(( insample_y, hist_exog.reshape(batch_size, windows_size, -1) ), dim=2)

        if self.futr_exog_size > 0:
            futr_exog = futr_exog.permute(0,2,1,3) # [B, C, W, L] -> [B, W, C, L]
            insample_y = torch.cat(( insample_y, futr_exog.reshape(batch_size, windows_size, -1) ), dim=2)

        if self.stat_exog_size > 0:
            stat_exog = stat_exog.unsqueeze(1).repeat(1, windows_size, 1) # [B, C] -> [B, W, C]
            insample_y = torch.cat(( insample_y, stat_exog ), dim=2)

        # GRU forward
        insample_y, _ = self.gru(insample_y)
        insample_y = self.adapterW(insample_y)
        
        return insample_y

In [ ]:
show_doc(GRU)

In [ ]:
show_doc(GRU.fit, name='GRU.fit')

In [ ]:
show_doc(GRU.predict, name='GRU.predict')

## Usage Example

In [ ]:
#| eval: false
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import matplotlib.pyplot as plt

from neuralforecast import NeuralForecast
from neuralforecast.losses.pytorch import MQLoss
from neuralforecast.utils import AirPassengersPanel, AirPassengersStatic
from neuralforecast.tsdataset import TimeSeriesDataset, TimeSeriesLoader

Y_train_df = AirPassengersPanel[AirPassengersPanel.ds<AirPassengersPanel['ds'].values[-12]] # 132 train
Y_test_df = AirPassengersPanel[AirPassengersPanel.ds>=AirPassengersPanel['ds'].values[-12]].reset_index(drop=True) # 12 test

fcst = NeuralForecast(
    models=[GRU(h=12,
                input_size=12,
                state_hsize=100,
                max_epochs=200,
                scaler_type='robust',
                futr_exog_list=['trend'],
                hist_exog_list=['y_[lag12]'],
                stat_exog_list=['airline1']),
    ],
    freq='M'
)
fcst.fit(df=Y_train_df, static_df=AirPassengersStatic)
forecasts = fcst.predict(futr_df=Y_test_df)

# Plot
plot_df = Y_test_df.copy()
plot_df['GRU'] = forecasts.reset_index(drop=False)['GRU']
plot_df = pd.concat([Y_train_df, plot_df])

plot_df = plot_df[plot_df.unique_id=='Airline1'].drop('unique_id', axis=1)
plt.plot(plot_df['ds'], plot_df['y'], c='black', label='True')
plt.plot(plot_df['ds'], plot_df['GRU'], c='blue', label='median')
plt.grid()
plt.legend()
plt.plot()